In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

In [2]:
dataset = load_dataset("csv", data_files="/home/jiangyunqi/Code/TangWen/Weibo-classification/response_v0.csv", split="train")

Using custom data configuration default-e54a9e9d07c50e9e


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/jiangyunqi/.cache/huggingface/datasets/csv/default-e54a9e9d07c50e9e/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


In [4]:
dataset[0]

{'review': '有八卦图的味道，有创意。忙活一上午饿了！连吃饭都忘记了！客人走了！想起来午饭没吃！还有昨天剩的饼！简单的做了素炒饼！填饱肚子在干活吧！鸡蛋纯是为了好看！鸡蛋是笨鸡蛋！',
 'label': '开心'}

In [9]:
label_list = dataset.unique("label")
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}
label2id
label_list

['开心',
 '惊讶',
 '困惑',
 '激动',
 '好奇',
 '期待',
 '不满',
 '挑衅',
 '伤心',
 '崇拜',
 '生气',
 '无语',
 '骄傲',
 '思考',
 '赞扬',
 '赞同',
 '疑问',
 '无助',
 '感激',
 '中性',
 '否定',
 '急切',
 '撒娇',
 '安慰',
 '害怕',
 '害羞',
 '不安',
 '道歉']

In [10]:

# datasets = dataset.train_test_split(test_size=0.1)
# datasets

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 1655
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 184
    })
})

In [11]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-large")

def process_function(examples):
    examples["label"] = [label2id[label] for label in examples["label"]]
    tokenized_examples = tokenizer(examples["review"], max_length=32, truncation=True, padding="max_length")
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True)
tokenized_datasets

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1655
    })
    test: Dataset({
        features: ['review', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 184
    })
})

In [12]:
tokenized_datasets['train'][0]

{'review': '奥特曼学堂：环性人格：此类人格障碍患者的心境一直在高涨夸大和抑郁悲观之间波动.这种有节律的心境变化相当规则,而且事前并无值得关注的外界诱因.这种人格障碍是环性情绪的一种变体,但多数环性人格者不会发展为环性情绪.环性人格被视为一种气质,许多有天赋和创造力的人都有表现.最后一句我喜欢',
 'label': 2,
 'input_ids': [101,
  1952,
  4294,
  3294,
  2110,
  1828,
  8038,
  4384,
  2595,
  782,
  3419,
  8038,
  3634,
  5102,
  782,
  3419,
  7397,
  4809,
  2642,
  5442,
  4638,
  2552,
  1862,
  671,
  4684,
  1762,
  7770,
  3885,
  1930,
  1920,
  1469,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': 2}

In [13]:
import os
os.environ['HTTP_PROXY'] = 'sgdo.yulki.codes:24396'
os.environ['HTTPS_PROXY'] = 'sgdo.yulki.codes:24396'

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-large")

Some weights of the model checkpoint at hfl/chinese-macbert-large were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [16]:
!pip install evaluate
import evaluate

# 如果网络不太好，也可以使用本地加载的方式
acc_metric = evaluate.load("accuracy")
f1_metirc = evaluate.load("f1")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 445.8 kB/s eta 0:00:00a 0:00:01


In [17]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

In [21]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=1,   # 训练时的batch_size
                               gradient_accumulation_steps=32,  # *** 梯度累加 ***
                               gradient_checkpointing=True,     # *** 梯度检查点 ***
                               optim="adafactor",               # *** adafactor优化器 *** 
                               per_device_eval_batch_size=1,    # 验证时的batch_size
                               num_train_epochs=1,              # 训练轮数
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True,
                               report_to=None)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=4,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=32,
gradient_checkpointing=True,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_toke

In [22]:
from transformers import DataCollatorWithPadding

# *** 参数冻结 *** 
for name, param in model.bert.named_parameters():
    param.requires_grad = False

trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

In [23]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Traceback (most recent call last):
  File "/home/jiangyunqi/anaconda3/envs/study-transformer/lib/python3.8/runpy.py", line 185, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/home/jiangyunqi/anaconda3/envs/study-transformer/lib/python3.8/runpy.py", line 144, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/home/jiangyunqi/anaconda3/envs/study-transformer/lib/python3.8/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/home/jiangyunqi/anaconda3/envs/study-transformer/lib/python3.8/site-packages/wandb/__init__.py", line 200, in <module>
    _sentry.setup()
  File "/home/jiangyunqi/anaconda3/envs/study-transformer/lib/python3.8/site-packages/wandb/analytics/sentry.py", line 40, in wrapper
    return func(self, *args, **kwargs)
  File "/home/jiangyunqi/anaconda3/envs/study-transformer/lib/python3.8/site-packages/wandb/analytics/sentry.py", line 76, in setup
    client = sent

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /home/jiangyunqi/anaconda3/envs/study-transformer/lib/python3.8/site-packages/transformers/train │
│ er.py:1645 in train                                                                              │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/jiangyunqi/anaconda3/envs/study-transformer/lib/python3.8/site-packages/transformers/train │
│ er.py:1862 in _inner_training_loop                                                               │
│                                                                                                  │
│   1859 │   │   self._globalstep_last_logged = self.state.global_step                             │
│   1860 │   │   model.zero_grad()                                                                 │
│   1861 │   │                                                                                     │
│ ❱ 1862 │   │   self.control = self.callback_handler.on_train_begin(args, self.state, self.contr  │
│   1863 │   │                                                                                     │
│   1864 │   │   # Skip the first epochs_trained epochs to get the random state of the dataloader  │
│   1865 │   │   if not args.ignore_data_skip:                                                     │
│                                                                                                  │
│ /home/jiangyunqi/anaconda3/envs/study-transformer/lib/python3.8/site-packages/transformers/train │
│ er_callback.py:353 in on_train_begin                                                             │
│                                                                                                  │
│   350 │                                                                                          │
│   351 │   def on_train_begin(self, args: TrainingArguments, state: TrainerState, control: Trai   │
│   352 │   │   control.should_training_stop = False                                               │
│ ❱ 353 │   │   return self.call_event("on_train_begin", args, state, control)                     │
│   354 │                                                                                          │
│   355 │   def on_train_end(self, args: TrainingArguments, state: TrainerState, control: Traine   │
│   356 │   │   return self.call_event("on_train_end", args, state, control)                       │
│                                                                                                  │
│ /home/jiangyunqi/anaconda3/envs/study-transformer/lib/pytho

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

# 模型预测

In [ ]:

sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

In [ ]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
pipe(sen)